<a href="https://colab.research.google.com/github/darkzard05/graph_neural_network/blob/main/graph_neural_network_by_pytorch_geometric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch-spline-conv -f https://data.pyg.org/whl/torch-1.10.0+cu111.html
!pip install torch_geometric

Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html
Looking in links: https://data.pyg.org/whl/torch-1.10.0+cu111.html


In [2]:
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear
import torch_geometric
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import APPNP, ARMAConv

print(torch.__version__)
print(torch.version.cuda)

1.10.0+cu111
11.1


In [3]:
dataset_name = 'CiteSeer'
dataset = Planetoid(root='/tmp/'+dataset_name, name=dataset_name)

In [4]:
data = dataset[0]
print(f'dataset: {dataset_name}')
print(f'number of nodes: {data.num_nodes}, number of edges: {data.num_edges}')
print(f'number of classes: {dataset.num_classes}, number of edge features: {data.num_edge_features}')
print(data.train_mask[:20])
print(data.train_mask.sum(), data.val_mask.sum(), data.test_mask.sum())

dataset: CiteSeer
number of nodes: 3327, number of edges: 9104
number of classes: 6, number of edge features: 0
tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True])
tensor(120) tensor(500) tensor(1000)


In [5]:
class Model(torch.nn.Module):
    def __init__(self, hidden, K, alpha):
        super().__init__()
        # self.layer1 = ARMAConv(dataset.num_features, hidden, dropout=0.75)
        # self.layer2 = ARMAConv(hidden, dataset.num_classes, dropout=0.75)
        self.layer1 = Linear(dataset.num_features, hidden)
        self.layer2 = Linear(hidden, dataset.num_classes)
        self.prop = APPNP(K=K, alpha=alpha)

    def reset_parameters(self):
        self.layer1.reset_parameters()
        self.layer2.reset_parameters()

    def forward(self, data):
        x, edge_index  = data.x, data.edge_index
        x = F.dropout(x, training=self.training)
        x = self.layer1(x).relu()
        x = F.dropout(x, training=self.training)
        x = self.layer2(x).relu()
        x = self.prop(x, edge_index)
        return x

In [6]:
print('cuda' if torch.cuda.is_available() else 'cpu', 'is available')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(hidden=256, K=50, alpha=0.1).to(device)
print(model)
model.reset_parameters()
print({i:list(j.shape) for i, j in model.named_parameters()})

data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    pred = model(data)
    optimizer.zero_grad(set_to_none=True)
    loss = criterion(pred[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss

def test(mask):
    model.eval()
    pred = model(data)
    optimizer.zero_grad(set_to_none=True)
    correct = pred.argmax(dim=1)[mask] == data.y[mask]
    acc = int(correct.sum()) / int(mask.sum())
    return acc

best_acc = 0
epochs = 100
for epoch in range(1, epochs+1):
    loss = train()
    train_acc = test(data.train_mask)
    val_acc = test(data.val_mask)
    test_acc = test(data.test_mask)
    if best_acc < test_acc:
        best_acc = test_acc
        print(f'epoch: {epoch} | loss: {loss.item():.4f} | train_acc: {train_acc:.4f} | val_acc: {val_acc} | test_acc: {test_acc}')

cuda is available
Model(
  (layer1): Linear(in_features=3703, out_features=256, bias=True)
  (layer2): Linear(in_features=256, out_features=6, bias=True)
  (prop): APPNP(K=50, alpha=0.1)
)
{'layer1.weight': [256, 3703], 'layer1.bias': [256], 'layer2.weight': [6, 256], 'layer2.bias': [6]}
epoch: 1 | loss: 1.7927 | train_acc: 0.6167 | val_acc: 0.426 | test_acc: 0.45
epoch: 2 | loss: 1.7109 | train_acc: 0.6333 | val_acc: 0.482 | test_acc: 0.489
epoch: 3 | loss: 1.5042 | train_acc: 0.7250 | val_acc: 0.528 | test_acc: 0.544
epoch: 4 | loss: 1.2607 | train_acc: 0.7417 | val_acc: 0.558 | test_acc: 0.581
epoch: 5 | loss: 1.0213 | train_acc: 0.7500 | val_acc: 0.556 | test_acc: 0.601
epoch: 6 | loss: 0.7942 | train_acc: 0.8250 | val_acc: 0.584 | test_acc: 0.612
epoch: 7 | loss: 0.5775 | train_acc: 0.9250 | val_acc: 0.66 | test_acc: 0.687
epoch: 11 | loss: 0.1784 | train_acc: 0.9667 | val_acc: 0.696 | test_acc: 0.69
epoch: 12 | loss: 0.1422 | train_acc: 0.9833 | val_acc: 0.68 | test_acc: 0.698
ep